**Autor**: Victor Teixidó López

In [2]:
import tensorflow as tf
from tensorflow import keras

# ME QUEDO CON TU CARA

Las Olivetti faces son un conjunto de datos clásico de reconocimiento de imágenes. Es una colección de 400 imágenes de 40 personas diferentes de tamaño 64×64. El objetivo es obtener un clasificador capaz de etiquetar correctamente las imágenes. Vamos a utilizar la librería Tensorflow de Google a través de la API Keras, que simplifica la definición de arquitecturas de redes neuronales.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np

## Apartado a

El primero paso va a ser obtener los datos. Los cogeremos de la librería scikit-learn a través de la función fetch_olivetti_faces que devolverá dos matrices de datos, una para los datos de entrada y otra para las etiquetas.

In [ ]:
from sklearn.datasets import fetch_olivetti_faces
Xn, yn = fetch_olivetti_faces(return_X_y=True);

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


### División de los datos

Dividimos los datos en conjuntos de entrenamiento y test (70%/30%).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xn, yn, train_size=0.7, random_state=1)

### Preprocessing

Después de dividir los datos en los conjuntos de entrenamiento y test, aplicaremos a ambos conjuntos un pequeño preproceso. El primer proceso consiste en normalizar los datos para que estén en el rango [0-1].

Debido a que Keras no distingue entre clasificación y regresión como lo hace scikit learn, dado que la tarea se define por la función de pérdida que se utiliza. Este es un problema multiclase, por lo que la función de pérdida que corresponde es la entropía cruzada categórica. Los problemas multiclase, se resuelven mediante una regresión multisalida que genera la probabilidad para cada clase mediante la función softmax. Es por esto que transformaremos las etiquetas del problema en una matriz con codificación one-hot.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def preprocessing(X, y):
    print('Tamaño original:{}'.format(X.shape))
    
    # normalization of the data [0-1]
    X = X / np.max(Xn)

    # target classes transformation
    encoder = OneHotEncoder(sparse='False')
    tarjet_encoding = encoder.fit(y.reshape(-1,1))

    y = tarjet_encoding.fit_transform(y.reshape(-1,1)).toarray()
    
    print('Nuevo tamaño:{}'.format(X.shape))
    return X, y

In [ ]:
X_train, y_train = preprocessing(X_train,y_train)
print()
X_test, y_test = preprocessing(X_test,y_test)

Tamaño original:(280, 4096)
Nuevo tamaño:(280, 4096)

Tamaño original:(120, 4096)
Nuevo tamaño:(120, 4096)


# Apartado b

El perceptrón multicapa utiliza capas totalmente conectadas (densas) para realizar cálculos. El tamaño de un perceptrón para imágenes puede ser muy grande. Es por esto que utilizaremos PCA para reducir el número de dimensiones a algo más razonable. Generaremos dos conjuntos de datos utilizando las primeras 10 y 20 componentes.

## PCA

In [ ]:
from sklearn.decomposition import PCA

pca_10 = PCA(n_components=10)
pca_20 = PCA(n_components=20)

Vamos a generar un conjunto de datos con las primeras 10 componentes obtenidas con pca.

In [ ]:
pca_10.fit(X_train);

X_train_10 = pca_10.transform(X_train);
X_test_10 = pca_10.transform(X_test)

PCA(n_components=10)

Vamos a generar otro conjunto de datos utilizando las primeras 20 componentes obtenidas con pca.

In [ ]:
pca_20.fit(X_train);

X_train_20 = pca_20.transform(X_train);
X_test_20 = pca_20.transform(X_test);

In [ ]:
print(X_train_10.size, ' ', X_test_10.size)
print(X_train_20.size, ' ', X_test_20.size)

2800   1200
5600   2400


El siguiente código define un perceptrón con una capa oculta con función de activación ReLU. Fíjate en que la capa de salida es un softmax de tamaño 40 que permite predecir las probabilidades de cada clase.

## Exploración parámetros

Vamos a crear perceptrones con un valor de entrada igual a 10 o 20, en función de que conjunto de datos estemos utilizando. Como salida tendremos un softmax de tamaño 40, lo que nos permite predecir las probabilidades de cada una de las distintas clases. Para la capa oculta exploraremos el compoartamiento en cuanto a porcentaje de acierto en función del número de neuronas utilizadas (25, 50, 75 o 100).

Una vez entrenado el modelo, predeciremos a que clase pertenece cada individuo del conjunto de test. Con el resultado, y a través de la función *np.argmax* veremos que ha predicho nuestro modelo. Comprobando esto con el conjunto *y_test* obtendremos el resultado de dicha red neuronal.

In [ ]:
input_shape = [10,20]
neurons = [25,50,75,100]
predictions_results = []

train = X_train_10
test = X_test_10
for x in input_shape:
  if x == 20:
    train = X_train_20
    test = X_test_20
  for y in neurons:
    # create the model
    model = keras.Sequential()
    model.add(keras.Input(shape=(x)))
    model.add(keras.layers.Dense(units=y, activation="relu"))
    model.add(keras.layers.Dense(40, activation="softmax"))

    # compile and fit the model
    model.compile(optimizer=keras.optimizers.SGD(), loss=tf.keras.losses.CategoricalCrossentropy());
    model.fit(train,y_train,batch_size=16, epochs=200,verbose=False);

    # get the results for each unit neurons
    pred = []
    pred = model.predict(test);
    predictions = pred.argmax(axis=1)

    predictions_results.append(predictions)

4/4 [==============================] - 0s 8ms/step


4/4 [==============================] - 0s 3ms/step


4/4 [==============================] - 0s 5ms/step


4/4 [==============================] - 0s 3ms/step


4/4 [==============================] - 0s 4ms/step


4/4 [==============================] - 0s 6ms/step


4/4 [==============================] - 0s 4ms/step


4/4 [==============================] - 0s 3ms/step


Definimos una función auxiliar para calcular los aciertos de un conjunto de predicciones dadas.

In [ ]:
def compute_accuracies(predictions):
  accuracies = []
  for i in range(0,len(predictions)):
    correct_elem = 0
    for j in range(0,predictions[i].size):

      if y_test[j].argmax()+1 == predictions[i][j]:
        correct_elem += 1
      
    accuracies.append((i,correct_elem/predictions[i].size))
    
  return accuracies

Vamos a ver los resultados de los distintos modelos entrenados.

In [ ]:
accuracies = compute_accuracies(predictions_results)

print('neurons 25-50-75-100 with 10 components:')
for x in accuracies:
  if x[0] == 4:
    print('\nneurons 25-50-75-100 with 20 components:')
  print(x[1])

neurons 25-50-75-100 with 10 components:
0.675
0.6916666666666667
0.6833333333333333
0.7333333333333333

neurons 25-50-75-100 with 20 components:
0.7833333333333333
0.7666666666666667
0.825
0.825


Podemos ver que parece haber un patrón tal que a mayor número de neuronas mejores resultados obtenemos en comparación a un menor número de estas. Como observación global, tanto con 10 como con 20 componentes, los mejores resultados se obtienen con 100 neuronas en la capa oculta.

Destacar que el mejor resultado se da en el modelo entrenado con 20 componentes y donde el número de neuronas en la capa oculta es igual a 75 o 100 (0.825 de acierto).

Un número bajo de neuronas puede derivas en *under-fitting* y, por el contrario, un gran número de neuronas puede conllevar un problema de *over-fitting*. Es por esto que, a pesar de que en nuestro caso ha sido así, no siempre un mayor número de neuronas corresponde a un mayor acierto en el conjunto testo.

# Apartado c

Una alternativa a los MLP son las capas convolucionales. Se trata de redes neuronales inspiradas en el funcionamiento de la corteza visual y especializadas en problemas de visión. Vamos a utilizar una capa convolucional para clasificar el conjunto de datos original. Como la entrada debe ser una matriz cuadrada, tendremos que transformar la forma de la matriz entrada.

In [ ]:
X_train = X_train.reshape((280,64,64))
X_test = X_test.reshape((120,64,64))

Las capas convolucionales se usan para procesar imágenes en color, por lo que asumen que cada imagen es una matriz 3D, la tercera dimensión es para los canales de color. En este caso, las imágenes son en escala de grises, por lo que tendremos que simular que tenemos una dimensión adicional. 

Vamos a utilizar la función reshape de numpy para transformar los datos de entrenamiento y test de tal manera que obtendremos como resultado una matriz de 4 dimensiones. La primera de estas son los ejemplos y las otras tres son la imagen de cada ejemplo.

In [ ]:
X_train = X_train.reshape(-1,64,64,1);
X_test = X_test.reshape(-1,64,64,1);

print(X_train.shape)

(280, 64, 64, 1)


En este caso vamos a explorar el número de neuronas de la capa convolucional fijando el paso de las convoluciones (stride) a 1 y el tamaño del kernel de las convoluciones a 3. El número de neuronas que exploraremos será de 1 a 10.

In [ ]:
neurons = [1,2,3,4,5,6,7,8,9,10]
predictions_results = []

for x in neurons:
  # create the model
  model = keras.Sequential()
  model.add(keras.Input(shape=(64,64,1,)))
  model.add(keras.layers.Conv2D(filters=x, kernel_size=3, strides=1, activation="relu"))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(40, activation="softmax"))

  # compile and fit the model
  model.compile(optimizer=keras.optimizers.SGD(), loss=tf.keras.losses.CategoricalCrossentropy());
  model.fit(X_train,y_train,batch_size=16, epochs=200,verbose=False);

  # get the results for each unit neurons
  pred = []
  pred = model.predict(X_test);
  predictions = pred.argmax(axis=1)

  predictions_results.append(predictions)

4/4 [==============================] - 0s 18ms/step


4/4 [==============================] - 0s 24ms/step


4/4 [==============================] - 0s 19ms/step


4/4 [==============================] - 0s 18ms/step


4/4 [==============================] - 0s 25ms/step


4/4 [==============================] - 0s 19ms/step


4/4 [==============================] - 0s 20ms/step


4/4 [==============================] - 0s 18ms/step


4/4 [==============================] - 0s 18ms/step


4/4 [==============================] - 0s 18ms/step


Al igual que hemos hecho en el apartado anterior, y a través del mismo método, vamos a calcular el acierto para cada uno de los modelos entrenados.

In [ ]:
accuracies = compute_accuracies(predictions_results)

for i in range(0,len(accuracies)):
  print('Neurons', i+1, ':', accuracies[i][1])

Neurons 1 : 0.875
Neurons 2 : 0.8833333333333333
Neurons 3 : 0.8833333333333333
Neurons 4 : 0.8916666666666667
Neurons 5 : 0.8833333333333333
Neurons 6 : 0.8833333333333333
Neurons 7 : 0.8916666666666667
Neurons 8 : 0.8916666666666667
Neurons 9 : 0.8916666666666667
Neurons 10 : 0.8916666666666667


Podemos ver como en los 10 modelos entrenados con capas convolucionales hemos obtenido mejores resultados que el mejor modelo del apartado anterior. Estos resultados no sorprenden ya que al fin y al cabo este tipo de modelos están especializados para trabajar la visión artificial. Son modelos pensados para trabajr con matrices de píxeles y que además permiten reduciar el coste computacional en gran medida.

A partir de las 7 neuronas alcanzamos un acierto de casi el 0.892. Tal y como decíamos antes, si tuviéramos que escoger un modelo nos quedaríamos con uno de estos debido al mejor acierto que nos proporcionan, más de un 6% mejor. Cualquier modelo con 7, 8, 9 o 10 neuronas de los vistos en este apartado, sería la mejor elección.



# Apartado d

El método summary() de la clase Model calcula cuántos parámetros tiene la red y también los parámetros por capa.

La forma de calcular los tamaños y la cantidad de parámetros a calcular en una red depende del tipo de capas con el que estemos trabajando, en nuestro caso capas densas. El output de cada capa se define en la declaración de la red, por otro lado, el número de parámetros se calcula con la siguiente formula.
$$
num\_param = num\_output * (num\_input + 1)
$$

El número de parámetros totales de una red, en nuestro caso, será la suma de los parámetros que hay en cada una de las redes.

Vamos a ver ahora el resumen de las capas que mejoes resultados nos han dado para cada uno de los tipos de redes con los que hemos trabajado. Un perceptrón multicapa y las capas convolucionales. El cálculo de los tamaños y los parámetros se obtienen internamente tal y como hemos explicado en el punto anterior.

In [ ]:
model = keras.Sequential(name='sequential')
model.add(keras.Input(shape=(20)))
model.add(keras.layers.Dense(units=100, activation="relu", name='dense_1'))
model.add(keras.layers.Dense(40, activation="softmax", name='dense_2'))

# summary also return the total number of parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 100)               2100      
                                                                 
 dense_2 (Dense)             (None, 40)                4040      
                                                                 
Total params: 6,140
Trainable params: 6,140
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = keras.Sequential(name='sequential')
model.add(keras.Input(shape=(64,64,1,)))
model.add(keras.layers.Conv2D(filters=10, kernel_size=3, strides=1, activation="relu", name='conv2d_1'))
model.add(keras.layers.Flatten(name='flatten_1'))
model.add(keras.layers.Dense(40, activation="softmax", name='dense_1'))

# summary also return the total number of parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 62, 62, 10)        100       
                                                                 
 flatten_1 (Flatten)         (None, 38440)             0         
                                                                 
 dense_1 (Dense)             (None, 40)                1537640   
                                                                 
Total params: 1,537,740
Trainable params: 1,537,740
Non-trainable params: 0
_________________________________________________________________


In [4]:
model = keras.Sequential(name='sequential')
model.add(keras.Input(shape=(64,64)))
model.add(keras.layers.Dense(units=100, activation="relu", name='dense_1'))
model.add(keras.layers.Dense(40, activation="softmax", name='dense_2'))

# summary also return the total number of parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64, 100)           6500      
                                                                 
 dense_2 (Dense)             (None, 64, 40)            4040      
                                                                 
Total params: 10,540
Trainable params: 10,540
Non-trainable params: 0
_________________________________________________________________


Primero de todo recordar que con el primer modelo obtuvimos un acierto en el conjunto test de un 0.825 y con el segundo modelo de un 0.892.

La principal, y clara, ventaja que tiene la segunda red respecto la primera es el porcentaje de acierto, al final, nuestro objetivo es conseguir crear un modelo capaz de predecir correctamente unas clases y por tanto, un mejor acierto siempre será mejor. La cuestión es, ¿és siempre rentable un mejor acierto? Podemos ver como el primero modelo tiene un número de parametros muchísimo más pequeño el segundo y esto acaba recayendo en que el la red convolucional, debido al alto número de parámetros, será mucho más costo que el primero, bastante más de hecho. En puntos como este, habría que debatir, en función del objetivo del problema, si perder algo de acierto es rentable en pos de un mejor rendimineto y menor costo computacional.

Si nuestra entrada para la red MLP hubieran sido los datos originales (datos 64x64 en vez de utilizar componentes principales) el tamaño de la primera capa hubiera variado. La segunda capa sería igual ya que el tamaño de entrada seguiría siendo 100 y la salida 40. La primera capa en cambio tendría un tamaño de 6500 parámetros.
$$
num\_param = num\_output * (num\_input + 1)
$$
$$
6500 = 100*(64+1)
$$

Por tanto, el tamaño total de la red pasaría de 6140 a 10540, casi el doble. Por tanto, era importante este caso utilizar los componentes principales en vez de los datos en su forma original.